### Overview

For this project I have chosen three cities (San Francisco, London and Paris that are known to be good places for gaming companies. The objective of the analysis is to check which of the three is better, and decide where to locate the new offices. 

Some of the functions used can be found in the .py files from the src field. However, due to technical problems I haven't been able to import some of them, so they have used directly in this notebook.

In [1]:
import pandas as pd
from pymongo import MongoClient
import math
from functools import reduce
import operator

In [2]:
# For working with API requests
import json, requests

In [3]:
# For visualization in maps

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [4]:
#selfmade functions 

import src.frame as fr
import src.maping as mp
import src.api as ap
import src.distance as dt

#### The project is divided in three sections: Mongodb, API and Visualization, and Final Analysis. In each section, the same processes are done with the three selected cities.

### Mongodb

In [5]:
#The first step is to import the data from the collection companies of Mongodb

In [6]:
conn = MongoClient("localhost:27017")

In [7]:
db = conn.get_database("ironhack")

In [8]:
companies = db.get_collection("companies")

In [9]:
#With this command we identify the fields of the collection, which will help us to define our next steps

companies.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

#### San Francisco 

In [10]:
def collection(city, lim):

    '''
    This function extracts information from those companies(documents) of a MongoDB collection that fulfill the following conditions:
        - The companies need to located in a specific city, which we set as argument depending in our interest.
        - They have to operate in one of this fields: analytics, games-video, mobile, software or web

    It takes as aruments the city we want to explore and the limit of obsevations we want
    
    The function returns an object with the name, city, latitude and lonitude, and category of the company.

    '''
    
    x = pd.DataFrame(companies.find({"offices.city": city, "$or":[{"category_code": "games-video"},
                                                        
                                                       {"category_code": "mobile"}, {"category_code": "software"}, 
                                                        
                                                       {"category_code": "web"}]}, {
    
    "name":1, "offices.city":1, "offices.latitude":1,
    "offices.longitude":1, "category_code":1}).limit(lim))
    
    return x

In [11]:
#We choose 5 companies from each city from which we are going to build the analysis up

#The collection function gets the information from the city and the number of observations we want as a dataframe

sf = collection("San Francisco", 6)

#One of the problems I had was that the latitude, longitude and city were all inside a same column.
#For this reason, I had to create a dataframe wiht this field separated. The city._df function fulfills this goal. 
#Both the collection and and city_df function can be found in the frame.py from the src field

sf_df = fr.city_df(sf, "San Francisco")

#There are some observations that have None values in latitude and longtiude, so we can delete them as they wont offer
#anyhting that we are interested in

sf_df = sf_df.dropna(0, how = 'all', thresh = 4)

sf_df

,name,category_code,city,latitude,longitude
0,StumbleUpon,web,San Francisco,37.775196,-122.419204
1,Wesabe,web,San Francisco,37.793148,-122.402567
2,Pownce,web,San Francisco,37.762541,-122.397224
4,Zooomr,web,San Francisco,37.775196,-122.419204
5,Pageflakes,web,San Francisco,37.758113,-122.414689


#### London

In [16]:
# Somo of the companies have nul values in latitude and/or longitude. This are companies I'm not interested in so I can
#delete them. The thing is, that because of this we need to change the limit of each city if we want to get five companies
#from each. That is the reason for using a limit of 13 and 6 for London a Paris, respectively

In [12]:
ln = collection("London", 13)

ln_df = fr.city_df(ln, "London")

ln_df = ln_df.dropna(0, how = 'all', thresh = 4)

ln_df

,name,category_code,city,latitude,longitude
6,Lastminute,web,London,37.090240,-95.712891
7,Wonga,web,London,51.519204,-0.162610
8,ITV,web,London,37.090240,-95.712891
9,Dopplr,web,London,51.523789,-0.087432
10,Jiglu,software,London,51.542360,-0.149522


#### Paris

In [13]:
pr = collection("Paris", 6)

pr_df = fr.city_df(pr, "Paris")

pr_df = pr_df.dropna(0, how = 'all', thresh = 4)

pr_df

,name,category_code,city,latitude,longitude
0,Netvibes,web,Paris,48.870806,2.346680
1,TVtrip,web,Paris,48.856667,2.350987
2,WatZatSong,web,Paris,48.866323,2.381514
3,Jooce,web,Paris,48.856667,2.350987
4,Trivop,web,Paris,48.855853,2.336575


### API and Visualization

In [14]:
def full_map(df, map_):

    '''
    As the previous function, this ones creates markers and inserts them in the map of our choosing using the information extracted from the API.

    In this case, as we just wanted to check for clubs, starbucks and airports, the funtion wont work fot other values. 

    It takes as arguments the dataframe in which we have stores all the API's information, and the map in which we want to insert the markers:
        - starbucks -> green and white coffe cup
        - airports -> black and white plane
        - clubs -> white and black glass

    '''
    
    for i,row in df.iterrows():
        
        if row["category"] == "starbucks":
            
            coffe = {
                "location" : [row["latitud"], row["longitud"]],
                "tooltip" : row["name"]
            }
            icon = Icon( color = "green",
                        prefix = "fa",
                        icon = "coffee",
                        icon_color = "white"
            )

            Marker (**coffe,icon = icon).add_to(map_)
        
        elif row["category"] == "airport" : 
            
            plane = {
                "location" : [row["latitud"], row["longitud"]],
                "tooltip" : row["name"]
            }
            icon = Icon( color = "black",
                        prefix = "fa",
                        icon = "plane",
                        icon_color = "white"
            )

            Marker (**plane,icon = icon).add_to(map_)
            
        elif row["category"] == "club" :
            
            party = {
                "location" : [row["latitud"], row["longitud"]],
                "tooltip" : row["name"]
            }
            icon = Icon( color = "white",
                        prefix = "fa",
                        icon = "glass",
                        icon_color = "black"
            )

            Marker (**party,icon = icon).add_to(map_)

    return map_

In [12]:
# First, we store our credentials in variables. As I haven't been able to install dotenv, I have the keys in .txt which I
#then inlclude in the gitignore so that my keys are not public

In [15]:
client_id = open("./keys.txt").readlines()[0]

In [16]:
client_secret = open("./keys.txt").readlines()[1]

### San Francisco

In [17]:
#This coordinates have been selected from one of the companies that we obtained in the previous step. 
#The objective is to set a map centered in one of the companies in order to center the analysis

initial_lat = 37.762541

initial_lon = -122.397224

In [18]:
#This function creates a map using the previous coordinates and a zoom of our choosing
#The documentation of this function can be found in the maping.py located in the src field

map_sf = mp.mapa(initial_lat, initial_lon, 15)

#### Starbuks

In [23]:
#The next step is to get data from the Foursquare API. The parametres we need are the client_id, client_secret and version,
#and the characteristics we want our search to be centered in: limit of findings, latitude and longitude (LL), 
#radius of search and a query, which is a word that describes what we are looking for, in this case, Starbuks. 

CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '37.762541, -122.397224'
RAD = 100
QUERY = 'StarBucks'

In [24]:
#We set the url to request the information from the API

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&query={QUERY}&limit={LIMIT}'

In [26]:
#This function obtains the data from the API and creates the object with which we are going to work later. 
#The documentation of this function can be found in the api.py from the src field

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&query={QUERY}&limit={LIMIT}'

starbucks = ap.extract(url)

In [28]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [32]:
def places_df(objeto, category, rango):

    '''
    This function is used with the information extracted from the API. 

    It only requires the object in which we have stored the information extracted (using the extract function from api.py).

    Using the getFromDict function and a for loop, it iterates through the different observations and extracts the values of the name, 
    latitude and longitude of each observation. In the following command lines, we can see the hierarchy that the functon follows to get the wanted value.

    The values are stored in a new list of dictionaries that is then turned into a df

    Also, a column named category is created. In this colum we can check the category of the value (club, starbucks, airport), something we will use
    for creating the map. 

    '''
    
    mapa_nombre = ["venue","name"]
    latitud = ["venue", "location","lat"]
    longitud = ["venue","location","lng"]

    x = []
    
    for diccionario in objeto:
        lista = {}
        lista["name"] = getFromDict(diccionario,mapa_nombre)
        lista["latitud"] = getFromDict(diccionario,latitud)
        lista["longitud"] = getFromDict(diccionario,longitud)
        
        x.append(lista)
        
        df = pd.DataFrame(x)
        
    c = [category for _ in range(rango)]
        
    df["category"] = c

    return df


In [33]:
#This function creates a dataframe based on the previous object. The columns are name, latitude, longitude and category

starbucks_sf = places_df(starbucks, "starbucks", 15)

In [101]:
starbucks_sf.head()

,name,latitud,longitud,category
0,Starbucks,37.767287,-122.393926,starbucks
1,Starbucks,37.765878,-122.403449,starbucks
2,Starbucks,37.770190,-122.403465,starbucks
3,Starbucks,37.771196,-122.389364,starbucks
4,Starbucks,37.766857,-122.409428,starbucks


#### Airports

In [35]:
#As in the case of starbucks, here we want to get the airports inside a certain radius. 
#Instead of using a keyword(query) for the search, we had to use a category_id that identifie the airport category. 
#There is a problem, and it is that this id can give back other objects apart from airports, so we should delete them

CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 5
LL = '37.762541, -122.397224'
RAD = 30000
CATEGORY_ID = '4bf58dd8d48988d1ed931735'

In [36]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

In [37]:
airports = ap.extract(url)

In [38]:
airports_sf = places_df(airports, "airport", 5)

In [39]:
airports_sf

,name,latitud,longitud,category
0,San Francisco International Airport (SFO) (San...,37.616713,-122.387094,airport
1,Oakland International Airport (OAK) (Oakland I...,37.711731,-122.212495,airport
2,OAK Runway,37.710480,-122.213753,airport
3,Hayward Executive Airport (HWD),37.662295,-122.120114,airport
4,The Centurion Lounge,37.617609,-122.387915,airport


#### Nightlife

In [40]:
#The same process as the previous one is carried out but with the categoyr_id of the nightclubs

CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '37.775196, -122.419204'
RAD = 30000
CATEGORY_ID = '4bf58dd8d48988d121941735'

In [41]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

In [42]:
clubs = ap.extract(url)

In [43]:
clubs_sf = places_df(clubs, "club", 15)

In [44]:
#once we have all three dataframes, we concatenate them in just one that we will use to create the map and its markers

frames = [starbucks_sf, airports_sf, clubs_sf]

san_francisco = pd.concat(frames)

In [45]:
def offices(df, mapa): 

    '''
    This function adds a marker for all the businesses that we have in our original df to the map created with the previous function (mapa()). 

    It takes as arguments the dataframe we want to use and the map in which we want to insert the markers (a lightblue and black briefcase). 

    This function is focused in seting the markers for the companies we extracted form the Mongodb collection.

    '''

    for i,row in df.iterrows():

            office = {
                "location" : [row["latitude"], row["longitude"]],
                "tooltip" : row["name"]
            }
            icon = Icon( color = "lightblue",
                        prefix = "fa",
                        icon = "briefcase",
                        icon_color = "black"
            )

            Marker (**office,icon = icon).add_to(mapa)

In [46]:
#The first step of filling the map we created with the mapa() function, is to insert the markers of the companies
#we obtained form the Mongodb collection 

offices(sf_df, map_sf)

In [103]:
#Then, we insert the markers of the places we obtained from Foursquare and we have the map created

full_map(san_francisco, map_sf)

### London

In [48]:
initial_lat = 51.523789
initial_lon = -0.087432

In [49]:
map_ln = mp.mapa(initial_lat, initial_lon ,15)

#### Starbucks London

In [50]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '51.500152, -0.087432'
RAD = 200
QUERY = 'StarBucks'

In [51]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&query={QUERY}&limit={LIMIT}'

In [52]:
starbucks = ap.extract(url)

In [53]:
starbucks_ln = places_df(starbucks, "starbucks", 15)

In [54]:
starbucks_ln.head()

,name,latitud,longitud,category
0,Starbucks,51.504796,-0.086873,starbucks
1,Starbucks,51.500896,-0.093399,starbucks
2,Starbucks,51.504041,-0.075553,starbucks
3,Starbucks,51.505487,-0.085133,starbucks
4,Starbucks,51.508436,-0.079160,starbucks


#### Airtports London

In [69]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 5
LL = '51.500152, -0.087432'
RAD = 30000
CATEGORY_ID = '4bf58dd8d48988d1ed931735'

In [70]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

In [71]:
airports = ap.extract(url)

In [72]:
airports_ln = places_df(airports, "airport", 5)

In [73]:
airports_ln

,name,latitud,longitud,category
0,London Heathrow Airport (LHR) (London Heathrow...,51.469408,-0.462348,airport
1,London City Airport (LCY) (London City Airport),51.503736,0.049638,airport
2,Biggin Hill Airport (BQH) (Biggin Hill Airport),51.331794,0.028845,airport
3,Elstree Aerodrome,51.654570,-0.323946,airport
4,Terminal 4,51.459342,-0.447328,airport


#### Nightlife 

In [74]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '51.500152, -0.087432'
RAD = 300
CATEGORY_ID = '4bf58dd8d48988d116941735'

In [75]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

In [76]:
clubs = ap.extract(url)

In [77]:
clubs_ln = places_df(clubs, "club", 6)

In [78]:
frames = [airports_ln, starbucks_ln, clubs_ln]

london = pd.concat(frames)

In [79]:
offices(ln_df, map_ln)

In [80]:
full_map(london, map_ln)

### Paris

In [81]:
initial_lat = 48.866323
initial_long = 2.381514

In [82]:
map_pr = mp.mapa(initial_lat, initial_long, 15)

#### Starbucks

In [83]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '48.866323, 2.381514'
RAD = 200
QUERY = 'StarBucks'

In [84]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&query={QUERY}&limit={LIMIT}'

In [85]:
starbucks = ap.extract(url)

In [86]:
starbucks_pr = places_df(starbucks, "starbucks", 15)

#### Airports

In [87]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 5
LL = '48.866323, 2.381514'
RAD = 30000
CATEGORY_ID = '4bf58dd8d48988d1ed931735'

In [88]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

In [89]:
airports = ap.extract(url)

In [90]:
airports_pr = places_df(airports, "airport", 5)

#### Nightlife


In [91]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '48.866323, 2.381514'
RAD = 30000
CATEGORY_ID = '4bf58dd8d48988d121941735'

In [92]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

In [93]:
clubs = ap.extract(url)

In [94]:
clubs_pr = places_df(clubs, "club", 15)

In [95]:
frames = [airports_pr, starbucks_pr, clubs_pr]

paris = pd.concat(frames)

In [96]:
offices(pr_df, map_pr)

In [97]:
full_map(paris, map_pr)

### Final Analysis

The first conclusion we extract from the visual analysis is that London can be excluded, as the companies that belong to the categories we are interested in are way to disperse and far away from the clubs and starbucks, which means our employees will not be able to enjoy themselves, something we absolutely cannot accept. Therefore, London is out of the way, and we have to choose between Paris or San Francisco.  

#### Paris vs San Francisco

For this section, I'm going to select one company of each city and decide in which point we should locate our new offices. The criteria used will be as follow:

    - The closest Starbucks. In this case we don't care if there are many, as long as there is one not too far away
    - The closest airport. The same reasoning as the Starbucks
    - Number of clubs near the office. For our employees, we shoul look for an area where there a certain number of clubs neraby, as diversity is key for entertainment. 
    - Number of companies close by. 
    
Following this reasoning, we are going to give a value to eah criteria. Fot starbucks and airports, we are going to give a different value depending on the distance. In the case of looking for numbers instead of clonseness, we will multiply the number os establishments that can be found in a range of 150 meters around the selected office, and multiply the number by a ratio. 

In [98]:
def haversine(coord1, coord2):

    '''
    This function calculates the distance between points. 

    The explanation of the formula is quiet complex, and it has been extracted from 'https://janakiev.com/blog/gps-points-distance-python/',
    an article of Medium where it is everything explained with detail. 
    
    '''

    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

#### The companies have chosen based on the visualization, checking that they were close to other companies, a Starbucks and a club, and somehow centered in the map and the coordinates

#### Zoomvr - San Francisco

In [99]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 5
LL = '37.775196,-122.419204'
RAD = 100
QUERY = 'StarBucks'

In [104]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&query={QUERY}&limit={LIMIT}'

len(starbucks)

starbucks = ap.extract(url)

closest_starbucks_sf = places_df(starbucks, "starbucks", 5)

In [105]:
closest_starbucks_sf

,name,latitud,longitud,category
0,Starbucks,37.777090,-122.417298,starbucks
1,Starbucks,37.777220,-122.419419,starbucks
2,Starbucks,37.769122,-122.428353,starbucks
3,Starbucks,37.784143,-122.407596,starbucks
4,Starbucks,37.782738,-122.431519,starbucks


In [102]:
#This command lines will give us the distance  between points. In this case we used one of the companies from the collection
#and calculated the distance between the company and the closest starbucks and airport

Zoomvr_coord = 37.775196,-122.419204
places = {
    'airport_OAK': (37.710480,-122.213753),
    'airport_INT': (37.711731,-122.212495),
    'Starbucks': (37.777090,-122.417298)
 
}

for place, coord in places.items():
    distance = haversine(Zoomvr_coord, coord)
    print(place, distance)

airport_OAK 19451.128191215263
airport_INT 19503.02684360474
Starbucks 269.17794670153245


In [149]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '37.775196,-122.419204'
RAD = 150
CATEGORY_ID = '4bf58dd8d48988d116941735'

In [152]:
# I have chosen a range of 150 metres of radius and a limit of 15 clubs. However, in the case of San Francisco there are only
# 7 companies that fulfill such criteria. 

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

clubs = ap.extract(url)

closest_clubs_sf = places_df(clubs, "club", 7)

In [151]:
len(closest_clubs_sf)

7

#### Netvibes - Paris

In [110]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 5
LL = '48.870806,2.346680'
RAD = 100
QUERY = 'StarBucks'

In [111]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&query={QUERY}&limit={LIMIT}'

starbucks = ap.extract(url)

closest_starbucks_pr = places_df(starbucks, "starbucks", 5)

In [112]:
closest_starbucks_pr

,name,latitud,longitud,category
0,Starbucks,48.870865,2.347020,starbucks
1,Starbucks France,48.869916,2.344444,starbucks
2,Starbucks,48.867229,2.347180,starbucks
3,Starbucks,48.871936,2.340611,starbucks
4,Starbucks,48.876078,2.344639,starbucks


In [114]:
Netvibes_coord = 48.870806,2.346680
places = {

    'airport_leBourget': (48.955101,2.441819),
    'airport_ORY': (48.728313, 2.365880),
    'Starbucks': (48.870865, 2.347020)
 
}

for place, coord in places.items():
    distance = haversine(Netvibes_coord, coord)
    print(place, distance)

airport_leBourget 11673.532058339944
airport_ORY 15911.2772019664
Starbucks 25.725511381572666


In [142]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180323'
LIMIT = 15
LL = '48.870806,2.346680'
RAD = 150
CATEGORY_ID = '4bf58dd8d48988d116941735'

In [143]:
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={LL}&radius={RAD}&limit={LIMIT}&categoryId={CATEGORY_ID}'

clubs = ap.extract(url)

closest_clubs_pr = places_df(clubs, "club", 15)


#### Conclusion 

From this results we can see that:

    - The company in San Francisco has only 7 clubs in a 150 meter radius, while the one in Paris has 15 (maybe more, as the limit set was 15). 
    - The closest Starbukcs is 27 meters away from the point in Paris, while in San Francisco it is located 270 metres away. 
    - Respect the airports, in Paris we can find two airports at a distance of 16km and 11.7km, while in San Francisco the two options are located 19km and 19.5km each. It is also important to mention that one of the airports in San Francisco, the OAK, is located on the othe side of the bay, which means our account managers will have to take a boat or something else rather than a cap to get there. 

San Francisco has always been known as the place of birth of enterpreneurship and startups, but there are other locations that are starting to take their place in this field, and Paris seems like a perfect place to start our own company, with many companies nearby from which our developers can learn and exchange experiences, and plenty of clubs and starbucks for our employees to enjoy themselves after a hard day of work. Also, facing the company's growth, it could be a better option to locate our offices in a city where the industry is growing, instead of doing it in a city where it is solidly established.